# **EDA**

En este notebook encontrarás el Análisis exploratorio de datos referente a nuestro proyecto. Para llevar a cabo un análisis de mercado decidimos limitar los Datasets a los Estados de: Nevada, Florida y Virginia.



### **Configuraciones Globales e Importaciones**
En esta sección, se instalan e importan todas las librerías y/o módulos necesarios para el proceso ETL (Extract, Transform, Load) y se establecen configuraciones globales de ser requerido. Se utilizan las siguientes librerías y herramientas:

In [1]:
#Se conecta Google Colaboratory con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instalamos SPARK para manejar grande volumnes de datos

In [2]:
#Instala pyspark en Google Colaboratory
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=1eee35b8236812443c50aac30ebeab7be454b9e2538c952ae29bac42d53dd6c1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
#Se importan las librerias
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import re
import os # Proporciona funciones para interactuar con el sistema operativo.
import requests # Se utiliza para realizar solicitudes HTTP.
import pandas as pd # Una librería de análisis de datos.
import seaborn as sns #S e utiliza para la visualización de datos.
import pyspark.pandas as ps # Proporciona una interfaz para trabajar con datos en Spark utilizando el formato de DataFrame de pandas.
import json # Se utiliza para trabajar con datos en formato JSON.
from pyspark.sql import SparkSession # Se utiliza para crear una instancia de SparkSession, que es la entrada principal para trabajar con Spark SQL.
from pyspark.sql import functions as F #  Proporciona funciones para trabajar con datos en Spark DataFrame.
from pyspark.sql.functions import array_contains # Esta función se utiliza para filtrar los datos basados en la presencia de un valor en un array.
from pyspark.sql.functions import sum, col # Se utiliza para acceder a una columna en un DataFrame de Spark.
from pyspark.sql.functions import split, substring, concat_ws, max, min, to_date
from pyspark.sql.functions import expr, regexp_replace, when
from pyspark.sql.types import StringType

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("EDA-exhaustivo")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

## **RUTA DE LOS ARCHIVOS**

### **DATASET: GOOGLE MAPS**

In [54]:
# Ruta a la carpeta que contiene los archivos CSV en Google Drive
carpeta = '/content/drive/MyDrive/Colab-Notebooks/transformaciones/meta-final.csv'

# Listar archivos en la carpeta
archivos = os.listdir(carpeta)

# Crear una lista para almacenar los DataFrames
df_list = []

# Iterar sobre los archivos y cargar cada archivo CSV en un DataFrame
for archivo_name in archivos:
    if archivo_name.endswith('.csv'):
        archivo_path = os.path.join(carpeta, archivo_name)
        df = spark.read.csv(archivo_path, header=True)
        # Convertir la columna "MISC" a tipo String si es necesario
        if 'MISC' in df.columns:
            df = df.withColumn("MISC", col("MISC").cast("string"))
        df_list.append(df)

# Unir los DataFrames en uno solo
df_metadata = df_list[0]
for dataframe in df_list[1:]:
    df_metadata = df_metadata.unionByName(dataframe)

In [34]:
# Ruta a la carpeta que contiene los archivos CSV en Google Drive
carpeta = '/content/drive/MyDrive/Colab-Notebooks/transformaciones/estados-limpios9.csv'

# Listar archivos en la carpeta
archivos = os.listdir(carpeta)

# Crear una lista para almacenar los DataFrames
df_list = []

# Iterar sobre los archivos y cargar cada archivo CSV en un DataFrame
for archivo_name in archivos:
    if archivo_name.endswith('.csv'):
        archivo_path = os.path.join(carpeta, archivo_name)
        df = spark.read.csv(archivo_path, header=True)
        # Convertir la columna "MISC" a tipo String si es necesario
        if 'MISC' in df.columns:
            df = df.withColumn("MISC", col("MISC").cast("string"))
        df_list.append(df)

# Unir los DataFrames en uno solo
df_Reviews_estados = df_list[0]
for dataframe in df_list[1:]:
    df_Reviews_estados = df_Reviews_estados.unionByName(dataframe)

### **DATASET: YELP**

In [8]:
business_yelp = pd.read_csv('/content/drive/MyDrive/Colab-Notebooks/transformaciones/bussi-limpio.csv')
checkin_yelp = pd.read_csv('/content/drive/MyDrive/Colab-Notebooks/transformaciones/checki-limpio.csv')
tip_yelp = pd.read_csv('/content/drive/MyDrive/Colab-Notebooks/transformaciones/tip-limpio.csv')
user_yelp = pd.read_csv('/content/drive/MyDrive/Colab-Notebooks/transformaciones/user-limpio.csv')

In [9]:
# Ruta a la carpeta que contiene los archivos CSV en Google Drive
carpeta = '/content/drive/MyDrive/Colab-Notebooks/Datos-Limpios/review-limpiowsen22.csv'

# Listar archivos en la carpeta
archivos = os.listdir(carpeta)

# Crear una lista para almacenar los DataFrames
df_list = []

# Iterar sobre los archivos y cargar cada archivo CSV en un DataFrame
for archivo_name in archivos:
    if archivo_name.endswith('.csv'):
        archivo_path = os.path.join(carpeta, archivo_name)
        df = spark.read.csv(archivo_path, header=True)
        # Convertir la columna "MISC" a tipo String si es necesario
        if 'MISC' in df.columns:
            df = df.withColumn("MISC", col("MISC").cast("string"))
        df_list.append(df)

# Unir los DataFrames en uno solo
df_Reviews_yelp = df_list[0]
for dataframe in df_list[1:]:
    df_Reviews_yelp = df_Reviews_yelp.unionByName(dataframe)

## #Esta función calcula el porcentaje de valores nulos que tiene cada columna de un determinado dataframe

In [10]:
# Cuenta el número de nulos en cada columna

def conteo_nulos(dataframe):
    # Construye expresiones de agregación para contar nulos en cada columna
    expresiones_agregacion = [sum(col(c).isNull().cast("int")).alias(c) for c in dataframe.columns]

    # Aplica las expresiones de agregación al dataframe
    conteo_nulos_por_columna = dataframe.agg(*expresiones_agregacion)

    # Muestra el resultado
    conteo_nulos_por_columna.show()

In [11]:
def contar_nulos(dataframe):

    # Obtener la cantidad de valores nulos por columna
  nulos_por_columna = dataframe.isnull().sum()

  print("Cantidad de valores nulos por columna:\n", nulos_por_columna)

## **VERIFICAMOS DATOS NULOS**

In [12]:
conteo_nulos(df_metadata)

+-------+----------+--------+-------+--------+---------+----+--------------+---+-----+----+
|address|avg_rating|category|gmap_id|latitude|longitude|name|num_of_reviews|url|state|city|
+-------+----------+--------+-------+--------+---------+----+--------------+---+-----+----+
|      0|         0|       0|      0|       0|        0|   0|             0|  0|    0|   0|
+-------+----------+--------+-------+--------+---------+----+--------------+---+-----+----+



In [35]:
conteo_nulos(df_Reviews_estados)

+-------+----+------+----+-------+----+---------+
|gmap_id|name|rating|text|user_id|date|sentiment|
+-------+----+------+----+-------+----+---------+
|      0|  12|    12|  12|     24|  24|       24|
+-------+----+------+----+-------+----+---------+



In [33]:
contar_nulos(business_yelp)

Cantidad de valores nulos por columna:
 Unnamed: 0      0
business_id     0
name            0
address         0
city            0
state           0
latitude        0
longitude       0
stars           0
review_count    0
attributes      0
categories      0
hours           0
dtype: int64


In [15]:
contar_nulos(checkin_yelp)

Cantidad de valores nulos por columna:
 business_id    0
date           0
dtype: int64


In [36]:
contar_nulos(tip_yelp)

Cantidad de valores nulos por columna:
 Unnamed: 0          0
user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64


In [37]:
contar_nulos(user_yelp)

Cantidad de valores nulos por columna:
 user_id               0
name                  0
review_count          0
useful                0
funny                 0
cool                  0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64


In [38]:

conteo_nulos(df_Reviews_yelp)

+---------+-------+-----------+------+------+----+-------+-------+---------+
|review_id|user_id|business_id|review|rating|cool| useful|  funny|sentiment|
+---------+-------+-----------+------+------+----+-------+-------+---------+
|        0|      0|          0|     0|     0|   0|2415413|2415413|  2415413|
+---------+-------+-----------+------+------+----+-------+-------+---------+



In [19]:
# Eliminar filas donde 'name' está vacío
df_Reviews_estados = df_Reviews_estados.dropna(subset=['name'])

In [20]:
# Eliminar filas donde 'user_id' está vacío
df_Reviews_estados = df_Reviews_estados.dropna(subset=['user_id'])

In [21]:
#Eliminamos los NULOS en 'address'
business_yelp = business_yelp.dropna(subset=['address'])

In [22]:
#Eliminamos los NULOS en 'text'
tip_yelp = tip_yelp.dropna(subset=['text'])

In [23]:
#Eliminamos los NULOS en 'name'
user_yelp = user_yelp.dropna(subset=['name'])

In [24]:
# Eliminar filas donde 'user_id' está vacío
df_Reviews_yelp = df_Reviews_yelp.dropna(subset=['user_id'])


In [25]:
# Eliminar filas donde 'cool' está vacío
df_Reviews_yelp = df_Reviews_yelp.dropna(subset=['cool'])


## **OUTLIERS**

Vamos a estudiar el intervalo de tiempo en el que transcurren las reseñas para ver si ocurre alguna inconsistencia.

In [40]:
# Encuentra el valor máximo y mínimo de 'date'
valor_maximo = df_Reviews_estados.select(max("date")).first()[0]
valor_minimo = df_Reviews_estados.select(min("date")).first()[0]

print(f'El último año es: {valor_maximo}')
print(f'El primer año es: {valor_minimo}')

El último año es: 2021-09-09
El primer año es: 1990-12-31


Hay reseñas desde 1990 hasta 2021 por parte de Google Maps, esto no puede ocurrir porque Google Maps existe desde 2005. Estos datos desde 1990 hasta 2005 ya se catalogan como resultados inesperados.

In [41]:
# Filtrar el DataFrame por la fecha '1990-12-31'
df_filtrado = df_Reviews_estados.filter(col('date') == '1990-12-31')


### Google compró esta compañía en octubre de 2004 para crear el popular software gratuito, basado en el navegador, Google Maps. Google Maps fue anunciado por primera vez en Google Blog el 8 de febrero de 2005.

In [42]:
# Filtrar el DataFrame por el año >= 2005
df_Reviews_estados = df_Reviews_estados.filter(col('date') >= 2005)

In [43]:
# Encuentra el valor máximo y mínimo de 'date'
valor_maximo = df_Reviews_estados.select(max("date")).first()[0]
valor_minimo = df_Reviews_estados.select(min("date")).first()[0]

print(f'El último año es: {valor_maximo}')
print(f'El primer año es: {valor_minimo}')

El último año es: None
El primer año es: None


Nuestros restaurantes pueden pertenecer a diferentes categorías, las cuales pueden ser muy variadas, hagamos un conteo del total de categorías que se encuentran en los datos.

In [46]:
# Crea una lista de todas las categorías de business_yelp
categories_yelp = business_yelp['categories'].str.split(',').explode().str.strip().unique()


In [47]:
import pandas as pd

# Convertir el DataFrame de Spark a pandas DataFrame
categories_metadata_pd = df_metadata.toPandas()

# Crea una lista de todas las categorías de df_metadata
categories_metadata = categories_metadata_pd['category'].str.split(',').explode().str.strip().unique()

# Une ambas listas y obtiene las categorías únicas
all_categories = pd.Series(list(categories_yelp) + list(categories_metadata)).unique()

print(f"Hay {len(all_categories)} categorías")


Hay 4265 categorías


Ahora vamos a encontrar las ciudades con mayor número de restaurantes, esto nos permitirá delimitar el análisis de mercado a las principales ciudades y zonas comerciales.

In [49]:
import pandas as pd

# Convertir el DataFrame de Spark a pandas DataFrame
city_counts_metadata_pd = df_metadata.toPandas()

# Cuenta el número de negocios por ciudad en df_metadata
city_counts_metadata = city_counts_metadata_pd['city'].str.strip().value_counts()

# Suma el número de negocios por ciudad de ambos DataFrames
combined_city_counts = city_counts_yelp.add(city_counts_metadata, fill_value=0)

# Obtiene las 10 ciudades con más negocios y las ordena
top_10_cities = combined_city_counts.sort_values(ascending=False).head(10)

# Imprime el resultado
print("Las 10 ciudades con más restaurantes son:")
print(top_10_cities)

Las 10 ciudades con más restaurantes son:
New York         20753.0
Los Angeles      17175.0
Brooklyn         16804.0
San Diego        10746.0
San Francisco     8407.0
Bronx             6693.0
Sacramento        5777.0
San Jose          5345.0
Queens            4460.0
Fresno            4353.0
Name: city, dtype: float64


Aquí nos chocamos con incongruencias en los datos, en el top 2 se encuentra la ciudad de Philadelphia, que como se verá a continuación a veces la ubica en el Estado de Florida y a veces en el Estado de Nevada; sin embargo en la realidad pertenece al Estado de Pensilvania.

Teniendo esto presente decidimos elegir para nuestro análisis de mercado las tres ciudades que si pertenecen a los estados que habiamos predefinido: Tampa, Miami Y Las Vegas. Siendo dos ciudades de Florida y una de Nevada en las que centraremos nuestra atención para iniciar el plan de expansión de nuestro cliente.

In [ ]:
#Imprimo los primeros 5 restaurantes que se encuentran en Philadelphia

business_yelp[business_yelp['city']=='Philadelphia'].head(5)

También es interesante conocer qué categorías de restaurantes tienen más negocio dentro de los tres Estados.

In [ ]:
# Concatenar las columnas 'categories' de ambos DataFrames
all_categories_series = pd.concat([metadata_sitios['category'], business_yelp['categories']], ignore_index=True)

# Dividir las listas de categorías y convertirlas en una Serie
all_categories_series = all_categories_series.str.split(',').explode().str.strip()

# Contar la frecuencia de cada categoría
category_counts = all_categories_series.value_counts()

# Obtener el top 5 de categorías más comunes
top_10_categories = category_counts.head(10)

# Imprimir el resultado
print("Top 10 categorías más comunes:")
print(top_10_categories)

En este caso nos quedamos con las cinco categorías que no son 'obvias': Fast food restaurant, Takeout Restaurant, Pizza restaurant, Nightlife, Sandwiches.

Queda por ver cuántos de los restaurantes son vegetarianos o veganos, para saber si tenemos suficientes datos.

In [ ]:

# Filtra las filas que contienen 'Vegetarian' o 'Vegan' en la columna 'category'
vegetarian_google = metadata_sitios[metadata_sitios['category'].str.contains('Vegetarian|Vegan', case=False)]

# Imprime el resultado
print(vegetarian_google.shape[0])

In [ ]:

# Filtra las filas que contienen 'Vegetarian' o 'Vegan' en la columna 'categories'
vegetarian_yelp = business_yelp[business_yelp['categories'].str.contains('Vegetarian|Vegan', case=False)]

# Imprime el resultado
print(vegetarian_yelp.shape[0])


Son datos suficientes para analizar competidores directos en el análisis de mercado. Sin embargo debemos extrar reseñas de nuestro cliente directamente de la API de Google Maps para llevar un control de los KPI.

Es importante explorar el rango que tiene el número de reseñas, esto con la finalidad de descartar posibles outlier. Como se verá a continuación el restaurante con más reseñas tiene 7671 lo cual es totalmente posible.

In [ ]:
# Encuentra el valor máximo y mínimo de 'num_of_reviews'
valor_maximo = metadata_sitios['num_of_reviews'].max()
valor_minimo = metadata_sitios['num_of_reviews'].min()

print(f'El mayor número de reseñas en google es: {valor_maximo}')
print(f'El menor número de reseñas en google es: {valor_minimo}')


In [ ]:
# Encuentra el valor máximo y mínimo de 'num_of_reviews'
valor_maximo = business_yelp['num_of_reviews'].max()
valor_minimo = business_yelp['num_of_reviews'].min()

print(f'El mayor número de reseñas en yelp es: {valor_maximo}')
print(f'El menor número de reseñas en yelp es: {valor_minimo}')


# **ANÁLISIS GRÁFICO**

Ahora vamos a tratar de visualizar tendencias y a interpretar gráficos, empezamos con dos histogramas que nos representan el progreso temporal que tuvieron las reseñas a través de los años en cada plataforma.

In [ ]:
#Creo una lista de los años en los que se hacen reseñas
all_years=[2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

In [ ]:
# Histograma para Google
plt.figure(figsize=(12, 6))
plt.hist(reviews_google['year'], bins=all_years, color='blue', alpha=0.7, label='Google',width=0.6, align='mid')
plt.xlabel('Año')
plt.xticks(np.arange(min(all_years), max(all_years) + 1, 1),fontsize=10)
plt.ylabel('Frecuencia')
plt.title('Histograma de cantidad de Reviews - Google')
plt.legend()
plt.show()

In [ ]:

# Histograma para Yelp
plt.figure(figsize=(12, 6))
plt.hist(reviews_yelp['year'], bins=all_years, color='red', alpha=0.7, label='Yelp',width=0.6, align='mid')
plt.xlabel('Año')
plt.xticks(np.arange(min(all_years), max(all_years) + 1, 1),fontsize=10)
plt.ylabel('Frecuencia')
plt.title('Histograma de cantidad de Reviews - Yelp')
plt.legend()
plt.show()


Como se puede observar hay un fuerte crecimiento en las reseñas, entre los años 2015 y 2019; desde el 2020 hay una tendencia a la baja debido a la pandemia y las consecuencias de la cuarentena preventiva.

In [ ]:
# Elimino los espacios laterales de las cadenas de texto en la columna 'city'
business_yelp['city'] = business_yelp['city'].str.strip()

metadata_sitios['city'] = metadata_sitios['city'].str.strip()

Como dije anteriormente, es mejor enfocarnos en las tres ciudades con mayor número de restaurante, ya que esto indica una economía dinámica y zonas comerciales atractivas. Esto ya entra en una etapa de 'pre-análisis de mercado', donde veremos que con los datos que tenemos podemos detectar patrones interesantes.

A continuación se realizarán mapas de calor de la presencia de restaurantes en las tres ciudades elegidas: Miami, Tampa y Las Vegas.

In [ ]:
#Muestro cuantos restaurantes de Miami hay en el Dataset business_yelp

business_yelp[business_yelp['city']=='Miami'].head(5)

In [ ]:
#Creo un dataframe para Miami

Miami_sitios=metadata_sitios[metadata_sitios['city']=='Miami']

In [ ]:
# Crea un mapa interactivo centrado en Miami con la librería folium
miami_map = folium.Map(location=[25.7617, -80.1918], zoom_start=11, control_scale=True)

# Crea un mapa de calor y lo agrega al mapa de Miami
heat_data = [[point.latitude, point.longitude] for point in Miami_sitios.itertuples()]
HeatMap(heat_data).add_to(miami_map)

#Visualiza el mapa
miami_map

Aquí se puede ver que nuestros datos son bastante coherentes, la zona más caliente es la que está más cerca de Miami Beach y las zonas más turísticas.

Repetimos el proceso para Tampa y para Las Vegas.

In [ ]:
business_yelp[business_yelp['city']=='Tampa'].head(5)

In [ ]:

metadata_sitios_tampa=metadata_sitios[metadata_sitios['city']=='Tampa'][['longitude','latitude']]
business_yelp_tampa=business_yelp[business_yelp['city']=='Tampa'][['longitude','latitude']]

Tampa_sitios = pd.concat([metadata_sitios_tampa, business_yelp_tampa], ignore_index=True).drop_duplicates()


In [ ]:
# Crea un mapa centrado en Tampa
tampa_map = folium.Map(location=[27.9506, -82.4572], zoom_start=11)  # Coordenadas aproximadas del centro de Tampa

# Crea un mapa de calor
heat_data = Tampa_sitios[['latitude', 'longitude']].values.tolist()
HeatMap(heat_data).add_to(tampa_map)

tampa_map

In [ ]:
business_yelp[business_yelp['city']=='Las Vegas']


In [ ]:

metadata_sitios[metadata_sitios['city']=='Las Vegas'].head(5)

In [ ]:

Vegas_sitios=metadata_sitios[metadata_sitios['city']=='Las Vegas']


In [ ]:


# Crea un mapa centrado en Las Vegas
vegas_map = folium.Map(location=[36.1699, -115.1398], zoom_start=11)  # Coordenadas aproximadas del centro de Las Vegas

# Crea un mapa de calor
heat_data_vegas = Vegas_sitios[['latitude', 'longitude']].values.tolist()
HeatMap(heat_data_vegas).add_to(vegas_map)

vegas_map

Por último queda ver si tenemos información suficiente para enfocarnos en nuestro nicho referente a la gastronomía vegetariana y/o vegana. Ya vimos que si tenemos una gran cantidad de restaurante, queda por ver las reseñas, para esto decidimos hacer dos histogramas parecidos a los que se hicieron anteriormente pero enfocándonos únicamente en restaurantes de la especialidad.

In [ ]:
#Crea listas de id de negocios vegetarianos en las dos plataformas
lista_vegetarianos_google=vegetarian_google['gmap_id'].tolist()
lista_vegetarianos_yelp=vegetarian_yelp['business_id'].tolist()

In [ ]:

#Filtra las reviews que pertenecen a alguno de los restaurantes enlistados
reviews_vegetarianas_google = reviews_google[reviews_google['gmap_id'].isin(lista_vegetarianos_google)]
reviews_vegetarianas_yelp = reviews_yelp[reviews_yelp['business_id'].isin(lista_vegetarianos_yelp)]


In [ ]:
# Histograma para restaurantes vegetarianos en Google Maps
plt.figure(figsize=(12, 6))
plt.hist(reviews_vegetarianas_google['year'], bins=all_years, color='green', alpha=0.7, label='Google',width=0.6, align='mid')
plt.xlabel('Año')
plt.xticks(np.arange(min(all_years), max(all_years) + 1, 1),fontsize=10)
plt.ylabel('Frecuencia')
plt.title('Histograma de cantidad de Reviews Vegetarianas - Google')
plt.legend()
plt.show()

In [ ]:
# Histograma para restaurantes vegetarianos en Yelp
plt.figure(figsize=(12, 6))
plt.hist(reviews_vegetarianas_yelp['year'], bins=all_years, color='green', alpha=0.4, label='Yelp',width=0.6, align='mid')
plt.xlabel('Año')
plt.xticks(np.arange(min(all_years), max(all_years) + 1, 1),fontsize=10)
plt.ylabel('Frecuencia')
plt.title('Histograma de cantidad de Reviews Vegetarianas - Yelp')
plt.legend()
plt.show()

Se aprecian tendencias similares a las de los restaurantes en general, esto indica que el consumo de la comida vegetariana incrementa y representa un gran nicho para invertir.

Los tips nos serán de utilidad para aprender de los errores de otros restaurantes y realizar estrategias más efectivas. Para esto debemos encontrar los tips que se hacen a los restaurantes vegetarianos o veganos y tener conocimiento de cuántos tenemos para un futuro análisis.

In [ ]:
tips_vegetarianos = tips_yelp[tips_yelp['business_id'].isin(lista_vegetarianos_yelp)]

In [ ]:
print(f"El número de tips a restaurantes vegetarianos es: {tips_vegetarianos.shape[0]}")